# Задания по SQL. Базовый уровень

## Задание 1 

Посчитайте, сколько компаний закрылось.

* SELECT count(id)
* FROM company
* WHERE status='closed'

## Задание 2 

Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы _company_. Отсортируйте таблицу по убыванию значений в поле _funding_total_. 

* SELECT funding_total
* FROM company
* WHERE country_code = 'USA' and category_code='news'
* ORDER BY funding_total DESC

## Задание 3

Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

* SELECT sum(price_amount)
* FROM acquisition
* WHERE term_code='cash' and acquired_at between '01.01.2011' and '31.12.2013'

## Задание 4

Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на _Silver_.

* SELECT first_name,
last_name,
twitter_username
* FROM people
* WHERE twitter_username LIKE 'Silver%'

## Задание 5

Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку _money_, а фамилия начинается на _K_.

* SELECT *
* FROM people
* Where twitter_username like '%money%' and last_name like 'K%'

## Задание 6

Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

* SELECT sum(funding_total),
country_code
* FROM company
* GROUP BY country_code
* ORDER BY sum(funding_total) DESC

## Задание 7 

Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

* SELECT CAST(funded_at AS date),
MIN(raised_amount),
MAX(raised_amount)
* FROM funding_round
* GROUP BY CAST(funded_at AS date)
* HAVING MIN(raised_amount)!=0 and MIN(raised_amount)!=Max(raised_amount)

## Задание 8 

Создайте поле с категориями:
* Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
* Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
* Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.

Отобразите все поля таблицы fund и новое поле с категориями.

* SELECT *,

* CASE 
WHEN  invested_companies>99 THEN 'high_activity'
WHEN  invested_companies>19 and invested_companies<100 then 'middle_activity'
ELSE 'low_activity'
END AS i

* FROM fund

## Задание 9 

Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

* SELECT
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       round(avg(investment_rounds))
* FROM fund
* group by activity
* order by round(avg(investment_rounds))

## Задание 10

* Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
* Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
* Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

* SELECT country_code,
min(invested_companies),
max(invested_companies),
avg(invested_companies)
* from fund
* WHERE CAST(founded_at AS date) between '2010-01-01' AND '2012-12-31'
* GROUP BY country_code
* HAVING MIN(invested_companies) <> 0
* ORDER BY AVG(invested_companies) DESC,country_code 
* LIMIT 10

## Задание 11

Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

* SELECT
    first_name,
    last_name,
    instituition
* FROM people AS i
* LEFT OUTER JOIN education AS e ON i.id = e.person_id

## Задание 12

Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

* SELECT c.name,
count(distinct ed.instituition)
* FROM company as c
* INNER JOIN people as p ON c.id=p.company_id
* INNER JOIN education as ed ON p.id=ed.person_id
* GROUP BY c.name
* ORDER BY COUNT(distinct ed.instituition) desc
* limit 5;

## Задание 13


Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

* SELECT  DISTINCT c.name
* FROM company AS c
* LEFT JOIN  funding_round AS f on f.company_id=c.id
* WHERE  c.status='closed'
and f.is_first_round=1
and f.is_last_round=1


## Задание 14

Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

* SELECT DISTINCT p.id
* FROM people AS p
* WHERE p.company_id IN 
        (SELECT DISTINCT c.id
        FROM company AS c
        JOIN funding_round AS fr
        ON c.id = fr.company_id
        WHERE c.status = 'closed'
        AND is_first_round = 1
        AND is_last_round = 1)


## Задание 15

Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

* SELECT DISTINCT p.id, e.instituition
* FROM people AS p
* JOIN education AS e ON p.id = e.person_id
* WHERE p.company_id IN 

        (SELECT DISTINCT c.id
        FROM company AS c
        JOIN funding_round AS fr
        ON c.id = fr.company_id
        WHERE c.status = 'closed'
        AND is_first_round = 1
        AND is_last_round = 1)

## Задание 16

Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

* SELECT DISTINCT p.id, count(e.instituition)
* FROM people AS p
* JOIN education AS e ON p.id = e.person_id
* WHERE p.company_id IN 

        (SELECT DISTINCT c.id
        FROM company AS c
        JOIN funding_round AS fr
        ON c.id = fr.company_id
        WHERE c.status = 'closed'
        AND is_first_round = 1
        AND is_last_round = 1)
* GROUP BY p.id

## Задание 17

Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

* SELECT AVG(total)
* FROM 
        (SELECT p.id, count(e.instituition) as total    
         FROM people AS p
         JOIN education AS e ON p.id = e.person_id
         WHERE p.company_id IN 

        (SELECT DISTINCT c.id
        FROM company AS c
        JOIN funding_round AS fr
        ON c.id = fr.company_id
        WHERE c.status = 'closed'
        AND is_first_round = 1
        AND is_last_round = 1)
        group by p.id) AS S1

## Задание 18

Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook

* SELECT avg(total_instituition)
* FROM 

            (SELECT p.id, 
             COUNT(e.instituition) AS total_instituition
             FROM people AS p JOIN education AS e ON  p.id = e.person_id
             WHERE company_id IN 
                 
                 (SELECT DISTINCT c.id
                  FROM company AS c
                  JOIN funding_round AS fr
                  ON c.id = fr.company_id
                  where c.name like 'Facebook')
                  group by p.id) AS S1

## Задание 19

* Составьте таблицу из полей:
_name_of_fund_ — название фонда;
_name_of_company_ — название компании;
_amount_ — сумма инвестиций, которую привлекла компания в раунде.
* В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

* SELECT f.NAME name_of_fund, c.NAME, fr.raised_amount amount
* FROM investment i
* JOIN company c ON i.company_id = c.id
* JOIN fund f ON i.fund_id = f.id
* JOIN funding_round fr ON fr.id = i.funding_round_id 
* WHERE c.milestones > 6 AND EXTRACT(YEAR FROM cast(fr.funded_at AS DATE)) IN (2012,2013)

## Задание 20

* Выгрузите таблицу, в которой будут такие поля:
            
     * название компании-покупателя;
     * сумма сделки;
     * название компании, которую купили;
     * сумма инвестиций, вложенных в купленную компанию;
     * доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций,округлённая до ближайшего целого числа. 
            
* Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
* Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

* SELECT cc.name, c.name, a.price_amount, cc.funding_total, round(a.price_amount/cc.funding_total)
* FROM acquisition as a
* INNER JOIN company as c on a.acquiring_company_id=c.id
* INNER JOIN company as cc on a.acquired_company_id=cc.id
* WHERE a.price_amount!=0 and cc.funding_total!=0
* ORDER BY a.price_amount DESC,cc.name
* LIMIT 10

## Задание 21

Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

* SELECT c.name AS social_c, EXTRACT(MONTH FROM CAST( funded_at AS DATE)) AS number_month
* FROM company AS c
* RIGHT OUTER JOIN funding_round AS fr ON c.id=fr.company_id
* WHERE c.category_code = 'social' AND funded_at BETWEEN '2010-01-01' AND '2013-12-31'
AND raised_amount >0;

## Задание 22

* Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
* количество уникальных названий фондов из США, которые инвестировали в этом месяце;
* количество компаний, купленных за этот месяц;
* общая сумма сделок по покупкам в этом месяце.

* WITH
* c_f AS 

       (SELECT EXTRACT (MONTH FROM CAST(fr.funded_at AS date)) AS month,
       COUNT(DISTINCT f.name) AS count_fund
       FROM funding_round AS fr
       RIGHT JOIN investment AS i ON fr.id=i.funding_round_id
       LEFT JOIN fund AS f ON  i.fund_id=f.id
       WHERE f.country_code = 'USA'
       AND EXTRACT (YEAR FROM CAST(fr.funded_at AS date)) IN (2010, 2011, 2012, 2013)
       GROUP BY month),
 
* c_c AS 

       (SELECT EXTRACT(MONTH FROM acquired_at) AS MONTH,
       COUNT(acquired_company_id) AS count_of_acquired,
       SUM(price_amount) AS sum_of_acquired
       FROM acquisition
       WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013
       GROUP BY MONTH)
 
* SELECT c_f.month AS month_funding_round,
       c_f.count_fund,
       c_c.count_of_acquired,
       c_c.sum_of_acquired
* FROM c_f JOIN c_c ON c_f.month=c_c.month    

## Задание 23

Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

* with
* a AS 

       (SELECT country_code, AVG(funding_total) AS year_2011
       FROM company
       WHERE EXTRACT(YEAR FROM founded_at)=2011
       GROUP BY country_code),

* b AS 

       (SELECT country_code, AVG(funding_total) AS year_2012
       FROM company
       WHERE EXTRACT(YEAR FROM founded_at)=2012
       GROUP BY country_code),

* c AS 

       (SELECT country_code, AVG(funding_total) AS year_2013
       FROM company
       WHERE EXTRACT(YEAR FROM founded_at)=2013
       GROUP BY country_code)

* SELECT a.country_code, a.year_2011, b.year_2012, c.year_2013
* FROM a
* JOIN b ON b.country_code=a.country_code
* JOIN c ON c.country_code=b.country_code
* ORDER BY year_2011 DESC